# Imports
Son las instalaciones necesarias para la preparacion de datos y la ejecucion del proyecto

In [2]:
%pip install Unidecode
%pip install num2words
%pip install spacy
%pip install stanza

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Aqui importamos las librerias necesarias y se descarga el modelo en español de stanza.
Stanza es una colección de herramientas precisas y eficientes para el análisis lingüístico de muchos lenguajes humanos. A partir del texto sin formato, Stanza lo divide en oraciones y palabras, y luego puede reconocer partes del discurso y entidades, realizar análisis sintácticos y más.

In [3]:
import pandas as pd
import re
from unidecode import unidecode
import num2words
import spacy
from spacy.lang.es import stop_words as stop_words
import stanza

In [4]:
stanza.download("es")

2025-02-20 18:31:01 INFO: Downloaded file to C:\Users\sebas\stanza_resources\resources.json
2025-02-20 18:31:01 INFO: Downloading default packages for language: es (Spanish) ...
2025-02-20 18:31:02 INFO: File exists: C:\Users\sebas\stanza_resources\es\default.zip
2025-02-20 18:31:06 INFO: Finished downloading models and saved to C:\Users\sebas\stanza_resources


Cargamos los datos que vamos a usar para el proyecto

In [26]:
df=pd.read_csv('fake_news_spanish.csv', encoding='utf-8', delimiter=';')
df

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [27]:
df['Titulo']=df['Titulo'].fillna('')
df['d']=df['Titulo']+df['Descripcion']
df


,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'The Guardian' va con Sánchez: 'Europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,El 'Ahora o nunca' de Joan Fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,Puigdemont: 'No sería ninguna tragedia una rep...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,El Defensor del Pueblo reclama a la Comunidad ...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,El EQUO plantea ceder la presidencia de la Com...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,Alberto Garzón: 'Que los Borbones son unos lad...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,Vox exige entrar en el Gobierno de Castilla y ...


# Preparacion de los datos

Se uso la libreria de stanza para tokenizar y lemmatizar la columand d del dataframe, debido a que este proceso se demora alrededor de una hora o más y es una cantidad excesiva de tiempo decidimos hacer una primera limpieza de los datos y exportarlo a un csv.

In [28]:
nlp = stanza.Pipeline(lang="es", processors="tokenize,lemma")

def tokenizar(linea):
    # dividir el texto en palabras
  
    texto_procesado = nlp(linea)
    lemmas = [word.lemma for sent in texto_procesado.sentences for word in sent.words]
        
    return lemmas

df['d']=df['d'].apply(tokenizar)
df
df['d']


2025-02-20 17:08:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-20 17:08:44 INFO: Downloaded file to C:\Users\sebas\stanza_resources\resources.json
2025-02-20 17:08:44 WARNING: Language es package default expects mwt, which has been added
2025-02-20 17:08:44 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

2025-02-20 17:08:44 INFO: Using device: cpu
2025-02-20 17:08:44 INFO: Loading: tokenize
2025-02-20 17:08:44 INFO: Loading: mwt
2025-02-20 17:08:44 INFO: Loading: lemma
2025-02-20 17:08:46 INFO: Done loading processors!


0        [', the, guardian, ', ir, con, Sánchez, :, ', ...
1        [revelan, que, el, GOBIERNO, negocio, el, libe...
2        [el, ', ahora, o, nunca, ', de, Joan, Fuster, ...
3        [Iglesias, alentar, a, yolanda, Díaz, ,, ERC, ...
4        [puigdemont, :, ', no, ser, ninguno, tragedia,...
                               ...                        
57058    [el, Defensor, de, el, Pueblo, reclamar, a, el...
57059    [el, equo, plantear, ceder, el, presidencia, d...
57060    [Alberto, Garzón, :, ', que, el, borbones, ser...
57061    [vox, exigir, entrar, en, el, Gobierno, de, ca...
57062    [uno, 300, persona, protestar, contra, el, vis...
Name: d, Length: 57063, dtype: object

Definimos la funcion de normalizar texto para eliminar caracteres invalidos en unidecode

In [30]:
def normalizar(texto):
    if isinstance(texto, str):

        return texto
    else:
        palabras = ""
        # todo a minúsculas
        for palabra in texto:
            palabra = palabra.lower()

            # tildes y diacríticas
            palabra = unidecode(palabra)
            palabras = palabras +","+ palabra

        return palabras


Se exporta el csv con la primera limpieza al archivo "dataframe_lematizado.csv"

In [31]:
df_2 = df.copy()
df_2

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[', the, guardian, ', ir, con, Sánchez, :, ', ..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revelan, que, el, GOBIERNO, negocio, el, libe..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[el, ', ahora, o, nunca, ', de, Joan, Fuster, ..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[Iglesias, alentar, a, yolanda, Díaz, ,, ERC, ..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[puigdemont, :, ', no, ser, ninguno, tragedia,..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[el, Defensor, de, el, Pueblo, reclamar, a, el..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[el, equo, plantear, ceder, el, presidencia, d..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[Alberto, Garzón, :, ', que, el, borbones, ser..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[vox, exigir, entrar, en, el, Gobierno, de, ca..."


In [38]:
df_2["d"]= df_2["d"].apply(normalizar)
type(df_2["d"][0])

str

In [ ]:
df.to_csv('dataframe_lematizado.csv',sep=";", index=False)

## Recarga el csv

Volvemos a cargar el dataframe desde el nuevo csv.

In [5]:
df_limpio=pd.read_csv('dataframe_lematizado.csv', encoding='utf-8', delimiter=';')
df_limpio

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,",',the,guardian,',ir,con,sanchez,:,',europa,ne..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,",revelan,que,el,gobierno,negocio,el,liberacion..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,",el,',ahora,o,nunca,',de,joan,fuster,sobre,el,..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,",iglesias,alentar,a,yolanda,diaz,,,erc,y,eh,bi..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,",puigdemont,:,',no,ser,ninguno,tragedia,uno,re..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,",el,defensor,de,el,pueblo,reclamar,a,el,comuni..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,",el,equo,plantear,ceder,el,presidencia,de,el,c..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,",alberto,garzon,:,',que,el,borbones,ser,uno,la..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,",vox,exigir,entrar,en,el,gobierno,de,castilla,..."


In [6]:
def string_to_list(line : str):
    return line.split(",")


In [7]:
df_limpio["d"] = df_limpio["d"].apply(string_to_list)
df_limpio

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[, ', the, guardian, ', ir, con, sanchez, :, '..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[, revelan, que, el, gobierno, negocio, el, li..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[, el, ', ahora, o, nunca, ', de, joan, fuster..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[, iglesias, alentar, a, yolanda, diaz, , , er..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[, puigdemont, :, ', no, ser, ninguno, tragedi..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[, el, defensor, de, el, pueblo, reclamar, a, ..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[, el, equo, plantear, ceder, el, presidencia,..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[, alberto, garzon, :, ', que, el, borbones, s..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[, vox, exigir, entrar, en, el, gobierno, de, ..."


In [11]:
df_limpio

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[, ', the, guardian, ', ir, con, sanchez, :, '..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[, revelan, que, el, gobierno, negocio, el, li..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[, el, ', ahora, o, nunca, ', de, joan, fuster..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[, iglesias, alentar, a, yolanda, diaz, , , er..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[, puigdemont, :, ', no, ser, ninguno, tragedi..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[, el, defensor, de, el, pueblo, reclamar, a, ..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[, el, equo, plantear, ceder, el, presidencia,..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[, alberto, garzon, :, ', que, el, borbones, s..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[, vox, exigir, entrar, en, el, gobierno, de, ..."


In [12]:
stopwords_espanol = stop_words.STOP_WORDS
#Se usa Spacy porque tiene un "vocabulario" más amplio que NLTK
#Genera un mejor resultado debido a esto

def stopwords_espanol_remove(line):
    new_line=[]
    for word in line:
        if word not in stopwords_espanol:
            new_line.append(word)
    return new_line


df_limpio['d']=df_limpio['d'].apply(stopwords_espanol_remove)
df_limpio

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[, ', the, guardian, ', sanchez, :, ', europa,..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[, revelan, gobierno, negocio, liberacion, mir..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[, ', ', joan, fuster, estatuto, valenciano, c..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[, iglesias, alentar, yolanda, diaz, , , erc, ..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[, puigdemont, :, ', tragedia, repeticion, ele..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[, defensor, pueblo, reclamar, comunidad, madr..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[, equo, plantear, ceder, presidencia, comunid..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[, alberto, garzon, :, ', borbones, ladron, hi..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[, vox, exigir, entrar, gobierno, castilla, le..."


In [24]:
def remove_disturb(line: list):
    new_line = []

    for word in line:
        if "'" in word:
            word = word.replace("'","")
        if word != "" and word != ":":
            new_line.append(word)
    return new_line

df_limpio['d']=df_limpio['d'].apply(remove_disturb)

In [25]:
def num_to_word(line):
    new_line=[]

    for word in line:
        if re.fullmatch(r'\d+',word):
            word=float(word)
            new_line.append(num2words.num2words(word, lang = 'es_CO'))
        else:
            new_line.append(word)
    return new_line

df_limpio['d']=df_limpio['d'].apply(num_to_word)
df_limpio

,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[the, guardian, sanchez, europa, necesitar, ap..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revelan, gobierno, negocio, liberacion, mirel..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[joan, fuster, estatuto, valenciano, cumplir, ..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[iglesias, alentar, yolanda, diaz, erc, eh, bi..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[puigdemont, tragedia, repeticion, eleccion, e..."
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[defensor, pueblo, reclamar, comunidad, madrid..."
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[equo, plantear, ceder, presidencia, comunidad..."
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[alberto, garzon, borbones, ladron, historicam..."
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[vox, exigir, entrar, gobierno, castilla, leon..."
